# Usage example showing how to determine which parameters are used for specific molecules

The Open Force Field Toolkit applies parameters based on the [SMARTS/SMIRKS language](http://www.daylight.com/dayhtml_tutorials/languages/smarts/). These examples are intended to expose how `ForceField` assigns SMIRKS-based parameters to test molecules.

<div class="alert alert-block alert-warning">
  <b>Note:</b> This API is not final, and we will be implementing a better way to visualize assigned parameters in the future.
</div>

### Example 1: Create an example molecule and label the force terms applied to it.

This example shows the parameters applied to a single molecule that is created from a SMILES string. 

The first output section consists of `Bond` parameters that would be assigned to the molecule. Each bond is described by the indices of the two atoms it connects, its "parameter ID", and the SMIRKS that caused it to match the target bond. 

The other `ParameterHandler`s print in order after that. 

Note that the `ToolkitAM1BCC` and `Electrostatics` handlers do not assign SMIRKS-based parameters, therefore they do not print any match information.

In [1]:
from openff.toolkit import ForceField, Molecule, Topology

# Create a simple molecule from SMILES and turn it into a topology.
molecule = Molecule.from_smiles("C=C(N)(C)")
topology = Topology.from_molecules([molecule])

# Let's label using the Sage force field
forcefield = ForceField("openff-2.0.0.offxml")

# Run the molecule labeling
molecule_force_list = forcefield.label_molecules(topology)

# Print out a formatted description of the parameters applied to this molecule
for mol_idx, mol_forces in enumerate(molecule_force_list):
    print(f"Forces for molecule {mol_idx}")
    for force_tag, force_dict in mol_forces.items():
        print(f"\n{force_tag}:")
        for atom_indices, parameter in force_dict.items():
            atomstr = ""
            for idx in atom_indices:
                atomstr += "%3s" % idx
            print(
                f"atoms: {atomstr}  parameter_id: {parameter.id}  smirks {parameter.smirks}"
            )

Forces for molecule 0

Constraints:
atoms:   0  4  parameter_id: c1  smirks [#1:1]-[*:2]
atoms:   0  5  parameter_id: c1  smirks [#1:1]-[*:2]
atoms:   2  6  parameter_id: c1  smirks [#1:1]-[*:2]
atoms:   2  7  parameter_id: c1  smirks [#1:1]-[*:2]
atoms:   3  8  parameter_id: c1  smirks [#1:1]-[*:2]
atoms:   3  9  parameter_id: c1  smirks [#1:1]-[*:2]
atoms:   3 10  parameter_id: c1  smirks [#1:1]-[*:2]

Bonds:
atoms:   0  1  parameter_id: b6  smirks [#6X3:1]=[#6X3:2]
atoms:   0  4  parameter_id: b85  smirks [#6X3:1]-[#1:2]
atoms:   0  5  parameter_id: b85  smirks [#6X3:1]-[#1:2]
atoms:   1  2  parameter_id: b8  smirks [#6X3:1]-[#7X3:2]
atoms:   1  3  parameter_id: b2  smirks [#6X4:1]-[#6X3:2]
atoms:   2  6  parameter_id: b87  smirks [#7:1]-[#1:2]
atoms:   2  7  parameter_id: b87  smirks [#7:1]-[#1:2]
atoms:   3  8  parameter_id: b84  smirks [#6X4:1]-[#1:2]
atoms:   3  9  parameter_id: b84  smirks [#6X4:1]-[#1:2]
atoms:   3 10  parameter_id: b84  smirks [#6X4:1]-[#1:2]

Angles:
atoms: 

### Example 2: Analyze the parameters assigned to groups of molecules

Here we analyze which parameters are applied to a group of molecules. 

This example uses `get_molecule_parameterIDs`, a simple utility function similar to `label_molecules`, but intended for use on large datasets. `get_molecule_parameterIDs` processes a list of molecules using a specified `ForceField` and returns the parameters that would be assigned to each molecule, grouping the results both by molecule and by parameter.

This example may be useful when adding new parameters, to ensure that they are applied to molecules in a given data set. 

This example also highlights the difference between "specific" and "generic" parameters. In the SMIRNOFF format, more than one SMIRKS-based parameter may match a motif in a molecule. Therefore, some parameters have precedence over others. Parameters which are "generic" are listed near the top of a parameter section, while more "specific" parameters occur near the end of the section. During parameter assignment, the furthest-down parameter that matches a motif in the molecule is assigned. 

During actual parameter assignment, if the toolkit is unable to assign SMIRKS-based parameters to any part of a molecule, an `UnassignedValenceParameterException` will be raised.

In [2]:
from openff.toolkit import ForceField, Molecule
from openff.toolkit.utils import get_molecule_parameterIDs

# The set of molecules that we want to parametrize in SMILES format.
molecule_smiles = [
    "[H][C@@]1([C@](OC1([H])[H])([H])O[H])O[H]",
    "[H]C1(C(C(C1([H])O[H])([H])[H])([H])O[H])[H]",
    "[H]C1(C(C(O1)([H])[H])([H])O[H])[H]",
    "[H][C@@]1(C([C@@](OC1([H])[H])([H])O[H])([H])[H])O[H]",
    "[H][C@@]1(C(O[C@](O1)([H])O[H])([H])[H])O[H]",
    "[H][C@]1(C([C@](O1)([H])O[H])([H])[H])O[H]",
    "[H][C@]1(C(OC(O1)([H])[H])([H])[H])O[H]",
    "[H][C@@]1([C@](C(C(C1([H])[H])([H])[H])([H])[H])([H])O[H])O[H]",
    "[H][C@]1([C@@](C(OC(C(C1([H])[H])([H])[H])([H])[H])([H])[H])([H])O[H])O[H]",
    "[H][C@@]1([C@](OC(C(C1([H])[H])([H])[H])([H])[H])([H])O[H])O[H]",
]

# Create the Molecule objects to parametrize.
mols = [Molecule.from_smiles(smiles) for smiles in molecule_smiles]

# Let's label using the original "Sage" force field
forcefield = ForceField("openff-2.0.0.offxml")

# This utility function creates dictionaries describing parameter assignment,
# grouped both by molecule and by parameter
parameters_by_molecule, parameters_by_ID = get_molecule_parameterIDs(mols, forcefield)

print("Molecules with parameter IDs:")
# parameters_by_ID is a dictionary where keys are parameter IDs and values
# are molecules in which each parameter ID occurs.
pids_ordered = sorted(list(parameters_by_ID.keys()))
for pid in pids_ordered:
    print(pid)
    for ids in parameters_by_ID[pid]:
        print("\t", ids)

Molecules with parameter IDs:
a1
	 [H][O][C@@]1([H])[O][C]([H])([H])[C@]([H])([O][H])[O]1
	 [H][O][C@]1([H])[O][C]([H])([H])[C@@]1([H])[O][H]
	 [H][O][C@@]1([H])[O][C@@]([H])([O][H])[C]1([H])[H]
	 [H][O][C@]1([H])[C]([H])([H])[C]([H])([H])[C]([H])([H])[C@@]1([H])[O][H]
	 [H][O][C@]1([H])[O][C]([H])([H])[C]([H])([H])[C]([H])([H])[C@@]1([H])[O][H]
	 [H][O][C@@]1([H])[C]([H])([H])[O][C]([H])([H])[C]([H])([H])[C]([H])([H])[C@]1([H])[O][H]
	 [H][O][C]1([H])[C]([H])([H])[O][C]1([H])[H]
	 [H][O][C@@]1([H])[O][C]([H])([H])[C@@]([H])([O][H])[C]1([H])[H]
	 [H][O][C@@]1([H])[O][C]([H])([H])[O][C]1([H])[H]
a2
	 [H][O][C@@]1([H])[O][C]([H])([H])[C@]([H])([O][H])[O]1
	 [H][O][C@]1([H])[O][C]([H])([H])[C@@]1([H])[O][H]
	 [H][O][C]1([H])[C]([H])([H])[C]([H])([O][H])[C]1([H])[H]
	 [H][O][C@@]1([H])[O][C@@]([H])([O][H])[C]1([H])[H]
	 [H][O][C@]1([H])[C]([H])([H])[C]([H])([H])[C]([H])([H])[C@@]1([H])[O][H]
	 [H][O][C@]1([H])[O][C]([H])([H])[C]([H])([H])[C]([H])([H])[C@@]1([H])[O][H]
	 [H][O][C@@]1([H])[C